In [ ]:
##*********************************************************
#      ROS quantity prediction (Keras+theano)  ver.6      *
#        copyright Masayuki Kogoshi                       * 
#                & Sumio Kogoshi, All right reserved      *
##*********************************************************
# -*- coding: utf-8 -*-
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing

#model name
savefile = "ROS_all_data.h5"]
#file name
name = "ROS_all_data"

# load of data
data_set = pd.read_csv(name + ".csv",sep=",",header=0)

#separation of input(x) and target(y)
x1 = pd.DataFrame(data_set.drop("quality",axis=1))


#SMOTE
#Extract evaluation value
yQ = pd.DataFrame(data_set["quality"])

sm = SMOTE(random_state=42)

#Amplify  minority data to the same number as the majority data
X_res, Y_res = sm.fit_sample(x1, yQ["quality"])

#Exclude the objective variable lnROS from the explanatory variables
x = pd.DataFrame(X_res.drop("lnROS",axis=1))
y = pd.DataFrame(X_res["lnROS"])


x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=50)

# standardization of x
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)


# input and output for keras
x_train = x_train.astype(np.float)

X = x_train
y = y_train

# number,dimensio,class_number
(n_samples, n_features) = X.shape

# 4 layers
model = Sequential()
# input layer
model.add(Dense(units=128, activation='relu', input_shape=(n_features,)))
# Dropout
model.add(Dropout(0.1))

# second layer
model.add(Dense(units=64, activation='relu'))
# Dropout
model.add(Dropout(0.1))

# third layer
model.add(Dense(units=32, activation='relu'))
# Dropout
model.add(Dropout(0.1))

# output layer
model.add(Dense(1))
model.summary

# compile
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# learning
history = model.fit(X, y, epochs=100, batch_size=128, verbose=2)

# save model
model.save_weights(savefile)

#loss_graph
plt.xlabel('time step')
plt.ylabel('loss')
plt.ylim(0,5)
loss, = plt.plot(history.history['loss'], c='#E69F00')
plt.show()

# Verify
x_test = scaler.transform(x_test)
y_pred = model.predict(x_test, verbose=0)

mse = mean_squared_error(y_test,y_pred)

####実行結果出力
y_test = np.array(y_test)
data1 = np.c_[y_test, y_pred]
data1 = pd.DataFrame(data1)
np.savetxt('out_' + name + '.csv', data1, delimiter=',')

plt.xlabel('target_lnROS')
plt.ylabel('predicted_lnROS')
plt.xlim(0,8)
plt.ylim(0,8)
plt.scatter(y_test,y_pred,s=1)
plt.show()

print("KERAS RMSE: %.2f" %(mse**0.5))
